In [ ]:
import pandas as pd
import numpy as np
import os
os.chdir(r'C:\Users\t0d00dh\01_Projects\2019\IDPs\walmart-realty-ms-case-study')

In [ ]:
hh = pd.read_csv('households.csv')
stores = pd.read_csv('stores.csv')
trips  = pd.read_csv('trips.csv')

## Structuring the data for the model

### 01.  Create matrix to store distances

In [ ]:
from math import sin, cos, sqrt, atan2, radians

# approximate radius of earth in km
R = 6373.0
def cor_distance ((lt1, ln1, lt2, ln2)):  
    lat1 = radians(lt1)
    lon1 = radians(ln1)
    lat2 = radians(lt2)
    lon2 = radians(ln2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c * 0.6213 # km to mile
    
    return (distance)

hh_new = pd.DataFrame()
hh_new ['hh_id'] =  hh['hh_id']
hh_new['hh_cor'] = zip(hh['home_latitude'],hh['home_longitude'])


stores['st_cor'] = zip(stores['Latitude'],stores['Longitude'])
vec = pd.DataFrame(stores['st_cor'])

# repeat the transpose vectore 
hh_new = hh_new.merge(pd.concat([vec.T]*len(hh_new),ignore_index=True), left_index=True, right_index=True)

hh_store = pd.DataFrame()
hh_store['hh_id'] =  hh['hh_id']
for i in range(28):
    hh_store[str(i)] = hh_new['hh_cor'] + hh_new.iloc[:,2:][i]
    
hh_store_distance = hh_store.iloc[:,1:].applymap(lambda x: cor_distance(x))



### 02. Rank the distances for each household

In [ ]:
## Original Ranking
hh_store_rank = hh_store_distance.apply(lambda x: x.rank(), axis=1)

## Normalized ranking
def normalize_rank (n,x):
    x1 = (x-1)/(n-1)
    return(x1)
hh_store_Nrank = hh_store_rank.applymap(lambda x: normalize_rank(28,x)) 

### 03. Transform the normalized ranking

In [ ]:
### Transform function

def trans_rank(p,x): # p is the power, adjustable
    x1 = 1-x**p
    return(x1)

# power = 0.15, 0.125,0.175,0.1

hh_store_TsRank = hh_store_Nrank.applymap(lambda x: trans_rank(0.15,x))
hh_store_TsRank_1 = hh_store_Nrank.applymap(lambda x: trans_rank(0.125,x))
hh_store_TsRank_2 = hh_store_Nrank.applymap(lambda x: trans_rank(0.175,x)) 
hh_store_TsRank_3 = hh_store_Nrank.applymap(lambda x: trans_rank(0.1,x)) 


# Create long table for each df
hh_store_distance['hh_id'] = hh['hh_id']
hh_store_rank['hh_id'] = hh['hh_id']
hh_store_Nrank['hh_id'] = hh['hh_id']
hh_store_TsRank['hh_id'] = hh['hh_id']
hh_store_TsRank_1['hh_id'] = hh['hh_id']
hh_store_TsRank_2['hh_id'] = hh['hh_id']
hh_store_TsRank_3['hh_id'] = hh['hh_id']

hhh_store_distance_long = pd.melt(hh_store_distance,id_vars=['hh_id'],var_name='Store_ID', value_name='Distance_Mile')
hh_store_rank_long = pd.melt(hh_store_rank,id_vars=['hh_id'],var_name='Store_ID', value_name='Ori_Rank')
hh_store_Nrank_long = pd.melt(hh_store_Nrank,id_vars=['hh_id'],var_name='Store_ID', value_name='Norm_Rank')
hh_store_TsRank_long = pd.melt(hh_store_TsRank,id_vars=['hh_id'],var_name='Store_ID', value_name='Ts_Rk_.15')
hh_store_TsRank_long_1 = pd.melt(hh_store_TsRank_1,id_vars=['hh_id'],var_name='Store_ID', value_name='Ts_Rk_.125')
hh_store_TsRank_long_2 = pd.melt(hh_store_TsRank_2,id_vars=['hh_id'],var_name='Store_ID', value_name='Ts_Rk_.175')
hh_store_TsRank_long_3 = pd.melt(hh_store_TsRank_3,id_vars=['hh_id'],var_name='Store_ID', value_name='Ts_Rk_.1')

# Merge All the dfs, for easy validation

df_dis_ranks = pd.merge(hhh_store_distance_long,hh_store_rank_long)
df_dis_ranks = pd.merge(df_dis_ranks,hh_store_Nrank_long)
df_dis_ranks = pd.merge(df_dis_ranks,hh_store_TsRank_long )
df_dis_ranks = pd.merge(df_dis_ranks,hh_store_TsRank_long_1 )
df_dis_ranks = pd.merge(df_dis_ranks,hh_store_TsRank_long_2 )
df_dis_ranks = pd.merge(df_dis_ranks,hh_store_TsRank_long_3 )

### 04. Calculate Store Visit Ratios

In [ ]:
trips['Store_ID'] = trips['store_choice']

In [ ]:
visit_freq = pd.pivot_table(trips,index =['hh_id'], columns=['Store_ID'],values = ['store_choice'],
                            aggfunc = 'count')
visit_freq = visit_freq.reset_index()
visit_freq.columns = ['hh_id' ,
                      0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]

visit_freq = visit_freq.fillna(0)
visit_freq['Total_Visit'] = visit_freq.iloc[:,1:].apply(np.sum,axis=1)

In [ ]:

Visit_Ratio = pd.DataFrame()

for i in range(len(visit_freq)):
    df = pd.DataFrame(list(visit_freq.iloc[i,1:-1]/visit_freq.iloc[i,-1])).T
    Visit_Ratio  = Visit_Ratio.append(df,ignore_index=True)
Visit_Ratio['hh_id'] = visit_freq['hh_id']

In [ ]:
# Convert the wide table to long table
Visit_Ratio_long = pd.melt(Visit_Ratio,id_vars=['hh_id'],var_name='Store_ID', value_name='Visit_Ratio')
Visit_Ratio_long['Store_ID'] = Visit_Ratio_long['Store_ID'].apply(lambda x: str(x))

In [ ]:
df_final = pd.merge(Visit_Ratio_long,df_dis_ranks)
stores['Store_ID'] = stores['Store_ID'].apply(lambda x: str(x))

df_final = pd.merge(df_final,stores[['Store_ID','Store_Category']],how = 'left', on ='Store_ID')

In [ ]:
df_final['Other'] = (df_final['Store_Category']=='Other').astype(int)
df_final['Specialty'] = (df_final['Store_Category']=='Specialty').astype(int)
df_final['Walmart Neighborhood Market'] = (df_final['Store_Category']=='Walmart Neighborhood Market').astype(int)
df_final['Walmart Supercenter'] = (df_final['Store_Category']=='Walmart Supercenter').astype(int)
df_final['Sams Club'] = (df_final['Store_Category']=='Sams Club').astype(int)

In [ ]:
df_final.to_csv('Store_HH_RK_Type_Long_1.csv')

In [ ]:
#df_final[df_final['hh_id'] == 0].to_csv('df_final_hh1.csv')

## Structuring the data for the model

In [ ]:
from sklearn import linear_model
import statsmodels.api as sm

In [ ]:
## ramdomly select 10000 household to train the model
import random
hh_train = random.sample(range(14150),10000)

train_bool = [ i in hh_train for i in df_final['hh_id']]
df_train =  df_final[train_bool]

test_ix = list(set(df_final.index) - set(df_train.index))
df_test = df_final.iloc[test_ix,:]


In [ ]:
Y = df_final['Visit_Ratio']
dic1 = {'T1':df_final['Ts_Rk_.1']*df_final['Specialty'],
 'T2':df_final['Ts_Rk_.1']*df_final['Walmart Neighborhood Market'],
 'T3':df_final['Ts_Rk_.1']*df_final['Walmart Supercenter'],
 'T4':df_final['Ts_Rk_.1']*df_final['Sams Club'],    
'T5':df_final['Ts_Rk_.1']*df_final['Other']}
X1 = pd.DataFrame(dic1)

In [ ]:
model = sm.OLS(Y, X1).fit()
predictions = model.predict(X1) 
 
print_model = model.summary()
print(print_model)

In [ ]:
dic = {'T1':df_train['Ts_Rk_.15']*df_train['Specialty'],
 'T2':df_train['Ts_Rk_.15']*df_train['Walmart Neighborhood Market'],
 'T3':df_train['Ts_Rk_.15']*df_train['Walmart Supercenter'],
 'T4':df_train['Ts_Rk_.15']*df_train['Sams Club'],    
'T5':df_train['Ts_Rk_.15']*df_train['Other']}

X = pd.DataFrame(dic)

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X, Y)

In [ ]:
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

In [ ]:
#X = sm.add_constant(X) # adding a constant
 
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)